### $n(z)$ Distributions

In [ ]:
import numpy as np
import matplotlib.pylab as plt

plt.rc('text', usetex=True)
plt.rc('font',**{'family':'sans-serif','serif':['Palatino']})
figSize  = (12, 8)
fontSize = 20

folder = '/home/harry/Desktop/MontePython-V3.4/data/KV450_COSMIC_SHEAR_DATA_RELEASE/REDSHIFT_DISTRIBUTIONS/'

In [ ]:
# Mean - Bayes
gfile_1 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Mean/Nz_Bayes_z0.1t0.3.asc')
gfile_2 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Mean/Nz_Bayes_z0.3t0.5.asc')
gfile_3 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Mean/Nz_Bayes_z0.5t0.7.asc')
gfile_4 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Mean/Nz_Bayes_z0.7t0.9.asc')
gfile_5 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Mean/Nz_Bayes_z0.9t1.2.asc')

# Samples - Bayes
s_gfile_1 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Bootstrap/Nz_Bayes_z0.1t0.3.asc')
s_gfile_2 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Bootstrap/Nz_Bayes_z0.3t0.5.asc')
s_gfile_3 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Bootstrap/Nz_Bayes_z0.5t0.7.asc')
s_gfile_4 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Bootstrap/Nz_Bayes_z0.7t0.9.asc')
s_gfile_5 = np.loadtxt(folder + 'Nz_Bayes/Nz_Bayes_Bootstrap/Nz_Bayes_z0.9t1.2.asc')

# Mean - DIR
dirf1 = np.loadtxt(folder + 'Nz_DIR/Nz_DIR_Mean/Nz_DIR_z0.1t0.3.asc')
dirf2 = np.loadtxt(folder + 'Nz_DIR/Nz_DIR_Mean/Nz_DIR_z0.3t0.5.asc')
dirf3 = np.loadtxt(folder + 'Nz_DIR/Nz_DIR_Mean/Nz_DIR_z0.5t0.7.asc')
dirf4 = np.loadtxt(folder + 'Nz_DIR/Nz_DIR_Mean/Nz_DIR_z0.7t0.9.asc')
dirf5 = np.loadtxt(folder + 'Nz_DIR/Nz_DIR_Mean/Nz_DIR_z0.9t1.2.asc')

In [ ]:
def plot_nz(gmean, gsamples, dirmean, label):
    plt.figure(figsize=(8,8))
    for i in range(1,gsamples.shape[1]):
        plt.plot(gsamples[:,0], gsamples[:,i], lw = 1)
    plt.plot(gmean[:,0], gmean[:,1], linestyle = 'dashed', c = 'k', lw = 2, label = 'Bayes (Mean)')
    plt.plot(dirmean[:,0], dirmean[:,1], linestyle = 'dotted', c = 'g', lw = 2, label = 'DIR (Mean)')
    plt.xlabel(r'$z$', fontsize = fontSize)
    plt.ylabel(r'$n_{'+str(label)+'}(z)$', fontsize = fontSize)
    plt.tick_params(axis='x', labelsize=fontSize)
    plt.tick_params(axis='y', labelsize=fontSize)
    plt.ylim(0.0, 5.5)
    plt.xlim(0.0, 6.0)
    plt.legend(loc = 'best',prop={'family':'sans-serif', 'size':15})
    plt.savefig('/home/harry/Desktop/Bayes-Plots/plots_'+str(label)+'.pdf', bbox_inches = 'tight')
    plt.close()

In [ ]:
plot_nz(gfile_1, s_gfile_1, dirf1, 1)
plot_nz(gfile_2, s_gfile_2, dirf2, 2)
plot_nz(gfile_3, s_gfile_3, dirf3, 3)
plot_nz(gfile_4, s_gfile_4, dirf4, 4)
plot_nz(gfile_5, s_gfile_5, dirf5, 5)

### Train a GP for $\sigma_{8}$

In [ ]:
import ml.zerogp as zgp
import utils.helpers as hp
import numpy as np

In [ ]:
inputs = hp.load_arrays('sigmaEight', 'cosmologies')
outputs = hp.load_arrays('sigmaEight', 'sigma_8')

In [ ]:
Nrestart    = 2
Ndim        = 5
bounds      = np.repeat(np.array([[-1.5,6]]), Ndim+1, axis = 0)
bounds[0] = np.array([-5, 5])

In [ ]:
gp_module = zgp.GP(inputs, outputs, 1E-6, True, False, False)

In [ ]:
gp_module.do_transformation()

In [ ]:
hyperparameters = gp_module.fit(method = 'L-BFGS-B', bounds = bounds, 
              options = {'ftol':1E-12, 'maxiter':500}, n_restart=Nrestart)

In [ ]:
testpoint = np.array([0.12, 3.45, 0.0225, 1.0, 0.72])

In [ ]:
gp_module.prediction(testpoint)

### Load MCMC samples

To compute the value of $\sigma_{8}$

In [ ]:
e_mulator = hp.load_pkl_file('samples', 'EMUGP_FF_Home_PC_Bayes_15000_18')
s_mulator = hp.load_pkl_file('samples', 'CLASS_FF_Home_PC_Bayes_15000_18')

In [ ]:
e_cosmologies = e_mulator.flatchain[:,0:5]
s_cosmologies = s_mulator.flatchain[:,0:5]

In [ ]:
nsamples_e = e_cosmologies.shape[0]
nsamples_s = s_cosmologies.shape[0]

In [ ]:
sigma_8_emu = [gp_module.prediction(e_cosmologies[i]) for i in range(nsamples_e)]
sigma_8_cls = [gp_module.prediction(s_cosmologies[i]) for i in range(nsamples_s)]

In [ ]:
sigma_8_emu = np.array(sigma_8_emu)
sigma_8_cls = np.array(sigma_8_cls)

### Compute $\Omega_{\textrm{m}}$

In [ ]:
omega_matter_emu = (e_cosmologies[:,0] + e_cosmologies[:,2])/e_cosmologies[:,4]**2
omega_matter_cls = (s_cosmologies[:,0] + s_cosmologies[:,2])/s_cosmologies[:,4]**2

### Triangle Plot

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from getdist import plots, MCSamples
import getdist
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

plt.rc('text', usetex=True)
plt.rc('font',**{'family':'sans-serif','serif':['Palatino']})
figSize  = (12, 8)
fontSize = 20

settings={'mult_bias_correction_order':0,'smooth_scale_2D':0.3, 'smooth_scale_1D':0.3}

In [ ]:
ln_As_emu = e_mulator.flatchain[:,1].reshape(nsamples_e, 1)
ln_As_cls = s_mulator.flatchain[:,1].reshape(nsamples_s, 1)

In [ ]:
samples_emu = np.concatenate((omega_matter_emu.reshape(nsamples_e, 1), ln_As_emu), axis = 1)
samples_cls = np.concatenate((omega_matter_cls.reshape(nsamples_s, 1), ln_As_cls), axis = 1)

In [ ]:
# samples_emu = np.concatenate((omega_matter_emu.reshape(nsamples_e, 1), sigma_8_emu), axis = 1)
# samples_cls = np.concatenate((omega_matter_cls.reshape(nsamples_s, 1), sigma_8_cls), axis = 1)

In [ ]:
# number of dimensions for plotting
ndim = 2

# some names for the parameters
names = ["x%s"%i for i in range(ndim)]

# actual labels
# labels = [r'$\Omega_{\textrm{m}}$', r'$\sigma_{8}$']
labels = [r'$\Omega_{\textrm{m}}$', r'$\textrm{ln}(10^{10}A_{\textrm{s}})$']

emu_plot = MCSamples(samples=samples_emu,names = names, labels = labels, settings = settings)
cls_plot = MCSamples(samples=samples_cls,names = names, labels = labels, settings = settings)

In [ ]:
c1 = '#EEC591'
c3 = '#8B0000'

legend_1 = mpatches.Patch(color=c1, label='Simulator')
legend_2 = mpatches.Patch(color=c3, label='Gaussian Process')
legend   = [legend_1,legend_2]

In [ ]:
G = plots.getSinglePlotter(width_inch=8, ratio=1)
G.settings.num_plot_contours = 2
G.settings.lw_contour = 2.5
G.settings.axes_fontsize = 25
G.settings.lab_fontsize = 25
G.settings.fontsize = 25 # important for padding in x-axis 
G.settings.alpha_filled_add = 0.6
emu_plot.updateSettings({'contours': [0.68, 0.95]})
cls_plot.updateSettings({'contours': [0.68, 0.95]})
G.plot_2d(cls_plot, 'x0', 'x1', filled=True, colors=[c1])
G.plot_2d(emu_plot, 'x0', 'x1', filled=True, colors=[c3])
plt.legend(handles=legend, loc = 'best',prop={'size':20}, borderaxespad=1)
# plt.savefig('plots/ln_As_omega_matter.pdf', transparent = False, bbox_inches = 'tight') 
plt.show()

### Testing Mask

In [ ]:
import cosmology.weaklensing as cw

In [ ]:
test = cw.model(emulator=False, ds=False)

In [ ]:
import setpriors as sp
import utils.common as uc
import numpy as np

In [ ]:
parameters = np.array([0.138, 2.766, 0.022, 1.05, 0.735, -0.83, -0.06*10**-4, 1.032, 1.143])

In [ ]:
# the cosmology part
cosmology = uc.mk_dict(sp.cosmo_names, parameters[0:5])

# the nuisance part
nuisance = uc.mk_dict(sp.nuisance_names, parameters[5:])

In [ ]:
xi = test.total_corr(cosmology, nuisance)

In [ ]:
maskings = np.split(test.mask, 30)

In [ ]:
xi_theory = test.spec_to_corr(cosmology, nuisance)

In [ ]:
test.theta_bins

In [ ]:
for i in range(15):
    xi_p = np.split(xi, 30)[i]*np.split(test.theta_bins, 2)[0]
    xi_m = np.split(xi, 30)[i+1]*np.split(test.theta_bins, 2)[1]
    
    print(xi_p)
    print('*'*100)
    print(xi_m)
    print('*'*100)

### HMCode test

In [1]:
import inference.likelihood as lk
import numpy as np 

In [2]:
test = lk.distributions(emulator = False, ds = False)

2022-02-12 14:25:58,964 | configurations | INFO | Angular scale-dependent c-term function loaded successfully.
2022-02-12 14:25:58,967 | configurations | INFO | Data loaded from directory: /home/harry/Desktop/MontePython-V3.4/data/KV450_COSMIC_SHEAR_DATA_RELEASE/DATA_VECTOR/KV450_xi_pm_files/
2022-02-12 14:25:59,011 | configurations | INFO | Covariance matrix, including shear calibration uncertainty loaded successfully
2022-02-12 14:25:59,012 | configurations | INFO | File for applying mask/cut loaded successfully.
2022-02-12 14:25:59,014 | configurations | INFO | The masked data vector is stored.
2022-02-12 14:25:59,014 | configurations | INFO | Mask applied to covariance matrix
2022-02-12 14:25:59,085 | configurations | INFO | Stored masked covariance matrix, including shear uncertainty, cut down to pre-specified scales.
2022-02-12 14:25:59,118 | configurations | INFO | Mask applied to data vector
2022-02-12 14:25:59,168 | configurations | INFO | Configurations for using the Bessel i

/Nz_Bayes/Nz_Bayes_Bootstrap_5000/Nz_Bayes_z0.1t0.3.asc
/Nz_Bayes/Nz_Bayes_Bootstrap_5000/Nz_Bayes_z0.3t0.5.asc
/Nz_Bayes/Nz_Bayes_Bootstrap_5000/Nz_Bayes_z0.5t0.7.asc


2022-02-12 14:25:59,630 | Bayes Redshifts | INFO | Bayes redshift distributions loaded sucessfully.
2022-02-12 14:25:59,633 | Bayes Redshifts | INFO | Redshift integrations performed at resolution of redshift distribution histograms


/Nz_Bayes/Nz_Bayes_Bootstrap_5000/Nz_Bayes_z0.7t0.9.asc
/Nz_Bayes/Nz_Bayes_Bootstrap_5000/Nz_Bayes_z0.9t1.2.asc


In [3]:
lk.sp.cosmo

{'omega_cdm': {'distribution': 'uniform', 'specs': [0.01, 0.34]},
 'ln10^{10}A_s': {'distribution': 'uniform', 'specs': [1.7, 3.3]},
 'omega_b': {'distribution': 'uniform', 'specs': [0.01875, 0.0075]},
 'n_s': {'distribution': 'uniform', 'specs': [0.7, 0.6]},
 'h': {'distribution': 'uniform', 'specs': [0.64, 0.18]}}

In [4]:
lk.sp.nuisance

{'A_IA': {'distribution': 'uniform', 'specs': [-6.0, 12.0]},
 'dc': {'distribution': 'norm', 'specs': [0.0, 0.0002]},
 'Ac': {'distribution': 'norm', 'specs': [1.01, 0.13]},
 'c_min': {'distribution': 'uniform', 'specs': [2.0, 1.13]}}

In [5]:
params = np.array([0.138, 2.766, 0.022, 1.05, 0.735, -0.83, -0.06*10**-4, 1.032, 2.143])

In [6]:
# test.loglikelihoodtest(params)

-91.60451787372688

In [6]:
# 1.204679e-01
# 3.073197e+00
# 2.191453e-02
# 9.669738e-01
# 6.687480e-01

# -3.175438e-01
# 2.133482e+00
# -7.151810e-05
# 9.062219e-01

# omega_cdm
# ln10^{10}A_s
# omega_b
# n_s
# h

# A_IA
# c_min
# dc
# Ac

# 94.1389

In [11]:
# params = np.array([1.204679e-01, 3.073197e+00, 2.191453e-02, 9.669738e-01, 6.687480e-01, -3.175438e-01, -7.151810e-05, 9.062219e-01, 2.133482e+00]) 

### Check for the likelihood 

In [30]:
import numpy as np 

In [31]:
mp_file = np.loadtxt('/home/harry/Desktop/MontePython-V3.4/chains/KV-450-Bayes/2022-02-12_100__1.txt')

In [32]:
mp_like = mp_file[:,1]
mp_samples = mp_file[:,2:11]
testpoints = mp_samples[:,[0, 1, 2, 3, 4, 5, 7, 8, 6]]
npoints = len(mp_like)

In [33]:
for i in range(npoints):
    print(f'MP: {mp_like[i]:.2f} and EMCEE {-test.loglikelihoodtest(testpoints[i]):.2f}')

MP: 94.61 and EMCEE 95.32
MP: 91.80 and EMCEE 92.66
MP: 91.69 and EMCEE 92.65
MP: 92.51 and EMCEE 91.71
